# MATH 405/607 

# Numerical Methods for Differential Equations

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)  [[course page]](https://github.com/cortner/math405_2022)

## PDEs Part 1: Diffusion

* Morton and Mayers: Numerical Solution of Partial Differential Equations
* todo -> add more references

In [ ]:
include("math405.jl")

### The Diffusion Equation

[source](https://www.dartmouth.edu/~cushman/courses/engs43/Chapter2.pdf)

<img src="control_volume.png" width="700">

* $[x, x + \Delta x] = $ "control volume"
* $x \in \mathbb{R}$ - infinite system (e.g. river, rod, blood vessel, ...)
* $c(x,t)$ = concentration (of a substance, temperature, ...)
* $q(x, t)$ = flux = rate of flow of the concentration through $x$

<img src="control_volume.png" width="500">

$$
\Delta x \cdot \frac{\partial c(x,t)}{\partial t} = \text{Import - Export} = q(x, t) - q(x+\Delta x, t)
$$

**Mass Balance:** Divide by $\Delta x$ and let $\Delta x \to 0$:
$$
\frac{\partial c(x,t)}{\partial t} = - \frac{\partial q(x, t)}{\partial x}
$$

<img src="control_volume.png" width="500">


$$
\frac{\partial c(x,t)}{\partial t} = - \frac{\partial q(x, t)}{\partial x}
$$

**Now need a constitutive law:**

**Pure Transport / Advection:** The fluid "transports" the substance with speed given by the fluid velocity $u$: 
$$
    q = u c \qquad \Rightarrow \qquad  \frac{\partial c}{\partial t} = - \frac{\partial}{\partial x} \big( u c \big).
$$

**Pure Diffusion:** Random motions of the fluid /  carrier cause the concentration to equilibrate locally, i.e.  the concentration will move from regions of high concentration to regions of low concentration. 
$$
  q = -  D \frac{\partial c}{\partial x} \qquad \Rightarrow \qquad  \frac{\partial c}{\partial t} = D \frac{\partial^2 c}{\partial x^2}
$$

**Linear Advection-Diffusion Equation:** 
$$
    \frac{\partial c}{\partial t} = - \frac{\partial}{\partial x}\big( u c \big) 
      + \frac{\partial}{\partial x}\Big( D(x) \frac{\partial c}{\partial x} \Big) 
$$
or in a bit more compact notation
$$
    c_t = - (uc)_x +  (D c_x)_x
$$

We will study advection next week. This week: diffusion!

#### E.g. heat conduction:

* Rod occupying the domain $[a, b]$
* $T(x)$ = temperature of rod at $x$
* $q = - k T_x$ is then called Fourier's law

$$
        T_t = k T_{xx}
$$

### Prototype diffusion equation

$$\begin{aligned} 
  u_t(t, x) &= u_{xx}(t,x) + f(t,x), \qquad t > 0, x \in (0, 1), \\ 
  u(t, 0) &= g_0(t), \\ 
  u(t, 1) &= g_1(t), \\ 
  u(0, x) &= u_0(x)
\end{aligned}$$

This is a prototype for parabolic PDE; one of the three most important classes of PDE (elliptic - see L12b, parabolic - this, hyperbolic - L14). We won't go into these technical details here, but many ideas and concepts we discuss for our prototype problems will apply in some form or other to more general PDEs.

To make our life a little easier, let's focus on 

$$\begin{aligned}
    u_t(t, x) &= u_{xx}(t, x) + f(x), \qquad t > 0, x \in (0, 1), \\  
    u(t, 0) &= u(t, 1) = 0, \\ 
    u(0, x) &= u_0(x).
\end{aligned}$$

With $f = 0$ this would, e.g., model heat flow in a rod with the end-points held at $T = 0$. (It also models overdamped dynamics of the displacement in a linearly elastic rod and a few other things ...)

I.e., there is no driving force and the system will converge to a steady state given by 

$$\begin{aligned}
    - \bar{u}_{xx} &= f(x),  \qquad x \in (0, 1),  \\
    \bar{u}(0) &= \bar{u}(1) = 0.
\end{aligned}$$

That is, $u(t, x) \to \bar{u}(x)$ as $t \to \infty$. (In what sense? How fast? Cf. exercises...)

Recall the discretisation of this scalar BVP in the introductory lectures. We started with the observation that 
$$
    \frac{u(x+h) - 2 u(x) + u(x-h)}{h^2}  = u_{xx}(x) + O(h^2)
$$

to obtain the discretised equation

$$ 
    - \frac{\bar{U}_{n+1} - 2 \bar{U}_n + \bar{U}_{n-1}}{h^2} = f(x_n).
$$

where $x_n = h n = n/N$. We then used a maximum principle (stability!) to prove that 

$$
    \max_n |\bar{U}_n - \bar{u}(x_n)| \leq C h^2.
$$

Today, we "simply" add time. Write $U_n(t) \approx u(t, x_n)$, $F_n = f(x_n)$ then we obtain 

$$\begin{aligned}
    \dot{U_n}(t) &= \frac{{U}_{n+1} - 2 {U}_n + {U}_{n-1}}{h^2} + F_n, \\ 
    U_n(0) &= u_0(x_n).
\end{aligned}$$

Now we can just use our favourite numerical integrator. This is called the "Method of Lines"

In [ ]:
MATH405.illustrate_mol()

In [ ]:
N = 50; 
L = Tridiagonal(ones(N-2), -2*ones(N-1), ones(N-2)) * N^2
X = range(0, 1, length=N+1)
U0 = X .* (1 .- X) .* exp.(3 * X)
plot(X, U0, size = (400, 150))

In [ ]:
using OrdinaryDiffEq
Tf = 1.0
prob = ODEProblem((U, p, t) -> L * U, U0[2:end-1], (0.0, Tf));
soln = solve(prob, RK4());

In [ ]:
@gif for t in 0.0:0.01:Tf 
    U = [[0.0]; soln(t); [0.0]]
    plot(X, U0, size = (500, 150), label = "u0")
    plot!(X, U, label = "t = " * (@sprintf("%.2f", t)), legend = :outertopright)
end

Everything works and we can just move on?

In [ ]:
# Let's analyze the numerical solution a bit...
dt = diff(soln.t)
plt1 = plot(soln.t[2:end], dt, yscale = :log10, size = (500, 300), lw = 3, label = "Δt", 
     legend = :outertopright, title = "# steps = $(length(soln.t))")

For such a simple problem, extremely smooth solution converging to a stable equilibrium this seems an excessive amount of work. Could this be a stiff problem? 

In [ ]:
# we can test this hypothesis by just trying a stiff solver!
stiff = solve(prob, Rosenbrock23());
plot!(plt1, stiff.t[2:end], diff(stiff.t), yscale = :log10, size = (400, 150), lw=3, label = "Δt (stiff)", 
     legend = :outertopright, title = "# steps = $(length(soln.t)) vs $(length(stiff.t))")

 So let's analyze this more carefully by looking at what happens when we apply the forward and backward Euler methods.

### Forward-Euler Discretisation

We have a spatial step $h$ and a time-step $\tau$, write $x_n = hn$, $t_m = \tau m$ and 

$$
    U_{n}^m \approx u(t_m, x_n)
$$

Then the *forward Euler* space-time discretisation of the IBVP is 

$$
    \frac{U_n^{m+1} - U_n^m}{\tau} = \frac{U_{n+1}^m - 2 U_{n}^m + U_{n-1}^m}{h^2} + F_n, \qquad n = 1, \dots, N-1.
$$

(note the boundary condition is implicit in $U_{0}^m = U_N^m = 0$!)

Let $u_n^m := u(m\tau, nh)$ then we argue again as in previous anlyses ...

$$ \begin{aligned} 
   \frac{U_n^{m+1} - U_n^m}{\tau} &= \frac{U_{n+1}^m - 2 U_{n}^m + U_{n-1}^m}{h^2} + F_n \\ 
   \frac{u_n^{m+1} - u_n^m}{\tau} &= \frac{u_{n+1}^m - 2 u_{n}^m + u_{n-1}^m}{h^2} + F_n + T_n^m, \\ 
\end{aligned} $$

where $T_n^m$ is the *truncation error*. 

Defining  $E_n^m := u_n^m - U_n^m$ and subtracting the two equations yields ... 

$$ \begin{aligned}
    \frac{E_n^{m+1} - E_n^m}{\tau}
    &= \frac{E_{n+1}^m - 2 E_{n}^m + E_{n-1}^m}{h^2} + T_n^m, \\
    |T_n^m| &\leq C \big(\tau \| u_{tt} \|_\infty + h^2 \|u_{xxxx} \|_\infty \big)
\end{aligned}$$

$$
\frac{E_n^{m+1} - E_n^m}{\tau}
    = \frac{E_{n+1}^m - 2 E_{n}^m + E_{n-1}^m}{h^2} + T_n^m
$$

Rearranging ... and setting $\mu := \tau / h^2$ ... 

$$ \begin{aligned}
    E_n^{m+1} 
    &=  E_n^m + \frac{\tau}{h^2} \big( E_{n+1}^m - 2 E_{n}^m + E_{n-1}^m \big) + \tau T_n^m \\ 
    &= \Big(1 - 2 \frac{\tau}{h^2}\Big) E_n^m + \frac{\tau}{h^2} (E_{n+1}^m + E_{n-1}^m) + \tau T_n^m
\end{aligned} $$

And now we can start estimating ...

$$
    |U_{n+1}^m| \leq \Big| 1 - 2 \frac{\tau}{h^2}\Big| |U_n^m| + \frac{\tau}{h^2} \big( |U_{n+1}^m| + |U_{n-1}^m| \big) + \tau \|T\|_\infty
$$

$$
    |E_{n}^{m+1}| \leq \Big| 1 - 2 \frac{\tau}{h^2}\Big| |E_n^m| + \frac{\tau}{h^2} \big( |E_{n+1}^m| + |E_{n-1}^m| \big) + \tau \|T\|_\infty
$$

Take $\max_n$ to obtain 

$$
    \| E^{m+1} \|_\infty \leq \Big| 1 - 2 \frac{\tau}{h^2}\Big| \|E^m\|_\infty + 2 \frac{\tau}{h^2}
        \|E^m\|_\infty + \tau \|T\|_\infty
$$

This grows exponentially in $k$ **UNLESS** we have $2 \tau h^{-2} \leq 1$. But under this restriction we have 

$$
    \Big| 1 - 2 \frac{\tau}{h^2}\Big| = \Big(1 - 2 \frac{\tau}{h^2}\Big)
$$

and we obtain ...

$$
    \| E^{m+1} \|_\infty \leq \|E^m\|_\infty  + \tau \|T\|_\infty \leq T_f \|T\|_\infty.
$$

**Theorem: [Convergence Forward-Euler]** Let $U_n^m$ be the solution of the forward-Euler discretisation, and suppose that $\tau \leq \frac{1}{2} h^2$, then 

$$
    \max_{n, m} \big| u(t_m, x_n) - U_n^m \big| \leq C T_f \big( \tau \| u_{tt} \|_\infty + h^2 \|u_{xxxx} \|_\infty \big).
$$

Remark: If $u_{xx}$ is replaced with $D u_{xx}$ then the restrictin becomes $\tau \leq \frac{D}{2} h^2$.

### Implicit Euler Discretisation

Remember the spatial semi-discretisation, 

$$
    \dot{U}_n(t) = \frac{U_{n+1}(t) - 2 U_n(t) + U_{n-1}(t)}{h^2} + F_n,
$$

then the *implicit Euler* space-time discretisation of the IBVP is 

$$
    \frac{U_n^{m+1} - U_n^m}{\tau} = \frac{U_{n+1}^{m+1} - 2 U_{n}^{m+1} + U_{n-1}^{m+1}}{h^2} + F_n, \qquad n = 1, \dots, N-1.
$$

We can now perform the same analysis as above, there are very few changes to the setup. Let $E_n^k = u(k\tau, n h) - U_n^k$ then 

$$ \begin{aligned}
    \frac{E_n^{m+1} - E_n^m}{\tau} 
    &= \frac{E_{n+1}^{m+1} - 2 E_{n}^{m+1} + E_{n-1}^{m+1}}{h^2} + T_n^m, \\ 
    |T_n^m| &\leq C\big( \tau \|u_{tt}\|_\infty + h^2 \|u_{xxxx}\|_\infty \big)
\end{aligned}$$

and rearraging yields 

$$
    \Big(1 + 2\frac{\tau}{h^2}\Big) E_n^{m+1}
    = E_n^m +  \frac{\tau}{h^2} \big( E_{n+1}^{m+1} + E_{n-1}^{m+1} \big) + \tau T_n^m
$$

Take moduli ... 

$$
    \Big(1 + 2\frac{\tau}{h^2}\Big) |E_n^{m+1}|
    \leq |E_n^m| +  \frac{\tau}{h^2} \big( |E_{n+1}^{m+1}| + |E_{n-1}^{m+1}| \big) + \tau |T_n^m|
$$

and $\max_n$ ... 

$$
    \Big(1 + 2\frac{\tau}{h^2}\Big) \|E^{m+1}\|_\infty
    \leq \|E^m\|_\infty + \frac{\tau}{h^2} \big( \|E^{m+1}\|_\infty + \|E^{m+1}\|_\infty \big) + \tau \|T\|_\infty
$$

The key difference now is that we can cancel the $\tau/h^2$ terms without any restrictions ...

$$
    \|E^{m+1}\|_\infty \leq \|E^m\|_\infty + \tau \|T\|_\infty \leq T_f \|T\|_\infty.
$$

... i.e. the implicit Euler discretisation is *unconditionally stable*.

**Theorem [Convergence Implicit Euler]:** Let $U_n^m$ be the solution of the implicit Euler discretisation then 

$$
    \max_{n, m} \big|u(t_m, x_n) - U_n^k\big| \leq C \big( \tau \|u_{tt} \|_\infty + h^2 \| u_{xxxx} \|_\infty \big).
$$



**Remarks:**

* Note that in fact 
$$
    u_{tt} = \partial_t \partial_{x}^2 u = \partial_x^2 \partial_t u = \partial_x^4 u.
$$
i.e. we can also write the error estimate as $ \dots \lesssim (\tau + h^2) \| u_{xxxx} \|_\infty$.
* If we want to balance errors then we might want to choose $\tau \approx h^2$ anyhow and we haven't gained too much. But if we are only after qualitative results, then this allows us to take large steps.
* In practise we would more likely choose at least a second-order time-stepping scheme. And we will see in the exercises that implicit schemes (can) inherit the stability of implicit Euler, while explicit schemes inherit the time-step restriction of explicit euler.

**Is the requirement $\tau \leq 0.5 h^2$ sharp?**

### Fourier Analysis

of the forward Euler method: now for $n \in \mathbb{Z}$ i.e. infinite chain $h \mathbb{Z}$ and without boundary condition.
$$
    \frac{U_{n}^{m+1} -  U_n^m}{\tau} = 
    \frac{U_{n+1}^m - 2 U_n^m + U_{n-1}^m}{h^2},  \qquad n \in \mathbb{Z}
$$

$$ 
    U_{n}^{m+1} = U_n^m + \mu \big( U_{n+1}^m - 2 U_n^m + U_{n-1}^m \big), \qquad \mu = \tau h^{-2}
$$

We can then use the semi-discrete Fourier transform: if $V = (V_n) \in \ell^2(\mathbb{Z})$ then 
$$ 
    \hat{V}(k) := \sum_{n \in \mathbb{Z}} V_n e^{-i k nh}, \qquad \text{for } k \in (-\pi/h, \pi/h].
$$

In the lecture on the SDFT we showed that 
$$
    V_n =  [ L U ]_n = U_{n+1} - 2 U_n + U_{n-1}
$$
then 
$$ 
    \hat{V}(k) = \widehat{LU}(k) = - 4 \sin^2( kh/2 ) \hat{U}(k)
$$

I.e. the Fourier transform *diagonalises* the finite-difference operator!

So if we apply the semi-discrete Fourier transform to 
$$ 
    U_{n}^{m+1} = U_n^m + \mu \big( U_{n+1}^m - 2 U_n^m + U_{n-1}^m \big).
$$
then we obtain 
$$ 
    \hat{U}^{m+1}(k) = \big[ 1 - 4 \mu \sin^2( kh/2 ) \big] \hat{U}^{m}(k).
$$

In [ ]:
plot(k -> 4 * sin(k/2)^2, -pi, pi, lw=3, size = (400, 300))

In [ ]:
μ = 0.51
Lhat(k) = 1 - 4 * μ  * sin(k/2)^2
plot(Lhat, -pi, pi, lw=3, size=(500, 300), label ="")

The multiplier $\hat{L}(k) := \big[ 1 - 4 \mu \sin^2( kh/2 ) \big]$ is monotone in $k$ so we need to check $k = 0$ and $k = \pi/h$. For $k = 0$ we just get $\hat{L}(0) = 1$, but for $k = \pi/h$, we have 
$$ 
    \hat{L}(\pi/h) = 1 - 4 \mu.
$$

This means that we obtain stability 
$$ 
    \| \hat{U}^{m+1} \|_{L^2} \leq \| \hat{U}^m \|_{L^2}
$$
(or, equivalently, $\| U^{m+1} \|_{\ell^2} \leq \|U^m \|_{\ell^2}$; cf Plancherel Theorem) if and only if 
$$
    |1 - 4 \mu| \leq 1 \qquad \Leftrightarrow \qquad \mu \leq 1/2  \qquad \Leftrightarrow \qquad \tau \leq  h^2 / 2
$$
I.e., precisely the condition we have before!

The first mode that becomes unstable is the highest-frequency mode:

In [ ]:
# use periodic boundary conditions to mimic the infinite chain analysis!
N = 100; h = 2*pi/N; tau = 0.55 * h^2; mu = tau / h^2; X = range(-pi+2*pi/N, pi, length = N)
U = exp.( - 10 * sin.(X).^2 ); # U = (1.0 .+ sign.(sin.(X))) / 2   # try this also 
L = spdiagm(-1 => ones(N-1), 0 => -2*ones(N), 1 => ones(N-1)); L[1, end] = 1; L[end, 1] = 1

@gif for n = 1:250  # 50 for the second example 
    global U, L, mu, X
    U += mu * (L * U)
    plot(X, U, lw = 2, label = "", size = (500, 300), ylims = (-0.2, 2.0), title = "n = $n")
end